Indices
Julia plots rows along the x axis and columns along the y axis.
This is opposite of python conventions, but inline with fits convetions.

Therefore in astronomy imaging conventions, the row (first, i) index corresponds to the x axis of the image, and the column (second, j) index corresponds to the y-axis.

I therefore use row, column order here which corresponds to plot x, y.


In [ ]:
using Optim
using CairoMakie
using FITSIO


In [ ]:
using Arya

In [ ]:
using Printf

In [ ]:
import StatsBase: median

In [ ]:
using ImageMorphology

In [ ]:
using ForwardDiff

In [ ]:
imgdir = "../yasone2/img_i_01/"
imgname = imgdir * "/nobkg.fits"
flagname = imgdir * "/flag.fits"
weightname = imgdir * "/flat_fielded.weight.fits"

In [ ]:
img = read_image(imgname)
img_flags = read_image(flagname)

img_masked = copy(img)
img_masked[img_flags .> 0] .= NaN;
img_err = read_image(weightname)
bkg_rms = median(img_err)

In [ ]:
function show_image(ax, img; log=false, kwargs...)

    if log
        img = copy(img)
        img[img .<= 0] .= NaN
        colorscale = log10
    else
        colorscale = identity
    end
    
    image(ax, img, interpolate=false, axis=(aspect=DataAspect(),); colorscale=colorscale, kwargs...)
end

In [ ]:
function show_image(img; log=false, kwargs...)

    if log
        img = copy(img)
        img[img .<= 0] .= NaN
        colorscale = log10
    else
        colorscale = identity
    end
    
    image(img, interpolate=false, axis=(aspect=DataAspect(),); colorscale=colorscale, kwargs...)
end

In [ ]:
show_image(img, log=true)

In [ ]:
img_labels = label_components(img_flags .== 1, strel_box((3, 3)))


In [ ]:
group_centroids, group_areas, group_bbox = get_group_centroids_areas(img_flags .== 1)

In [ ]:
function get_cutouts(img, centroids)
    cutouts = Vector{Array{eltype(img),2}}()

    Ny, Nx = size(img)
    cutout_size = 50
    
    for cen in centroids    
        # centroid is Float64, convert to Int
        xcen = round(Int, cen[1])
        ycen = round(Int, cen[2])
    
        # maximum allowed radius without going out of bounds
        r = min(
            cutout_size,
            xcen - 1,
            ycen - 1,
            Nx - xcen,
            Ny - ycen,
        )
    
        if r <= 0
            r = 0
        end
    
        xmin = xcen - r
        xmax = xcen + r
        ymin = ycen - r
        ymax = ycen + r
    
        push!(cutouts, img[xmin:xmax, ymin:ymax])
    end

    return cutouts
end

In [ ]:
cutouts_label = get_cutouts(img_labels, group_centroids)

In [ ]:
length(unique(img_labels))

In [ ]:
cutouts_err = get_cutouts(img_err, group_centroids)

In [ ]:
cutouts = get_cutouts(img_masked, group_centroids)

In [ ]:
for i in 100:150
    f = show_image(cutouts_label[i])
    nx, ny = size(cutouts_label[i])
    # scatter!(nx/2, ny/2)
    bbox = group_bbox[i]
    println(bbox)
    cen = group_centroids[i]

    println(i, " ", group_areas[i], " ", sum(img_labels .== i))
    println(unique(cutouts_label[i]))
    println(findfirst(sum(img_labels .== i, dims=2) .> 0)[1] - cen[1])
    println(findlast(sum(img_labels .== i, dims=2) .> 0)[1] - cen[1])
    println(findfirst(sum(img_labels .== i, dims=1) .> 0)[2] - cen[2])
    println(findlast(sum(img_labels .== i, dims=1) .> 0)[2] - cen[2])

    x1, x2 = nx/2 + bbox[1], nx/2 + bbox[2]
    y1, y2 = ny/2 + bbox[3], ny/2 + bbox[4]
    
    lines!([x1, x2, x2, x1, x1], [y1, y1, y2, y2, y1], color=COLORS[2])

    display(f)
end

# Model Fitting

In [ ]:
iimg_rand

In [ ]:
x_img

In [ ]:
iimg_rand
Nx, Ny = size(iimg_rand)
x = collect(1:Nx)
y = collect(1:Ny)
x_img = repeat(x, 1, Ny)
y_img = repeat(y', Nx, 1)

f = show_image(iimg_rand)

scatter!(vec(x_img), vec(y_img), color=vec(y_img), markersize=3)
f

In [ ]:
"""
Fit a Moffat PSF to an image cutout by minimizing chi-squared.

Optimized parameters:
(scale, dx, dy, alpha, gamma)
"""
function fit_moff_psf_chi2(
    image::AbstractMatrix;
    uncertainty = nothing,
    mask = nothing,
    initial_scale::Real = 1.0,
    initial_shift::Tuple{Real,Real} = (0.0, 0.0),
    bounds = nothing,
    sat_level = 62_000,
)

    # --- Handle NaNs ---
    image_clean = copy(image)
    finite_mask = isfinite.(image_clean)

    # --- Combine masks ---
    combined_mask = mask === nothing ? finite_mask : (finite_mask .& .!mask)

    # --- Degrees of freedom ---
    n_valid = count(combined_mask)
    n_params = 5
    dof = max(1, n_valid - n_params)

    Nx, Ny = size(image_clean)

    # --- Coordinate grids ---
    x = collect(1:Nx)
    y = collect(1:Ny)
    x_img = repeat(x, 1, Ny)
    y_img = repeat(y', Nx, 1)

    x_c = (Nx) / 2
    y_c = (Ny) / 2

    # --- Bounds ---
    if bounds === nothing
        bounds = ((-Nx/3, Nx/3), (-Ny/3, Ny/3))
    end

    lower = [0.0, bounds[1][1], bounds[2][1], 0.0, 0.0, 0.0, 0.0]
    upper = [Inf, bounds[1][2], bounds[2][2], 10, 20, 0.99, 2π]

    # --- Chi² function ---
    function chi2_func(p)
        scale, dx, dy, alpha, gamma, ell, theta = p

        model = moffat_2d(
            x_img .- dx .- x_c,
            y_img .- dy .- y_c;
            amplitude = scale,
            alpha = alpha,
            gamma = gamma,
            ellipticity=ell,
            theta=theta,
        )

        model = min.(model, sat_level)

        resid = image_clean .- model
        return sum(abs2, resid[combined_mask] ./ uncertainty[combined_mask])
    end

    # --- Initial guess ---
    p0 = [
        initial_scale,
        initial_shift[1],
        initial_shift[2],
        2.5,
        2.0,
        0.01, 
        0.01,
    ]

    # --- Optimization ---
    opt = optimize(
        chi2_func,
        lower,
        upper,
        p0,
        Fminbox(LBFGS()),
        Optim.Options(show_trace = false),
        autodiff = :forward
    )

    pbest = Optim.minimizer(opt)
    min_chi2 = Optim.minimum(opt)

    best_scale, best_dx, best_dy, best_alpha, best_gamma, best_ell, best_theta = pbest

    # --- Best-fit model ---
    best_model = moffat_2d(
        x_img .- best_dx .- x_c,
        y_img .- best_dy .- y_c;
        amplitude = best_scale,
        alpha = best_alpha,
        gamma = best_gamma,
        ellipticity = best_ell,
        theta = best_theta
    )

    best_model = min.(best_model, sat_level)
    residual = image_clean .- best_model

    return Dict(
        :scale => best_scale,
        :shift => (best_dx, best_dy),
        :alpha => best_alpha,
        :gamma => best_gamma,
        :theta => best_theta,
        :ellipticity => best_ell,
        :chi2 => min_chi2,
        :reduced_chi2 => min_chi2 / dof,
        :model => best_model,
        :residual => residual,
        :success => Optim.converged(opt),
        :n_valid_pixels => n_valid,
        :dof => dof,
    )
end


In [ ]:
"""
Fit a Moffat PSF to an image cutout by minimizing chi-squared.

Optimized parameters:
(scale, dx, dy, alpha, gamma)
"""
function fit_moff_simple_psf_chi2(
    image::AbstractMatrix;
    uncertainty = nothing,
    mask = nothing,
    initial_scale::Real = 1.0,
    initial_shift::Tuple{Real,Real} = (0.0, 0.0),
    bounds = nothing,
    sat_level = 62_000,
    alpha = 2.6,
)

    # --- Handle NaNs ---
    image_clean = copy(image)
    finite_mask = isfinite.(image_clean)

    # --- Combine masks ---
    combined_mask = mask === nothing ? finite_mask : (finite_mask .& .!mask)

    # --- Degrees of freedom ---
    n_valid = count(combined_mask)
    n_params = 5
    dof = max(1, n_valid - n_params)

    Nx, Ny = size(image_clean)

    # --- Coordinate grids ---
    x = collect(1:Nx)
    y = collect(1:Ny)
    x_img = repeat(x, 1, Ny)
    y_img = repeat(y', Nx, 1)

    x_c = (Nx) / 2
    y_c = (Ny) / 2

    # --- Bounds ---
    if bounds === nothing
        bounds = ((-Nx/3, Nx/3), (-Ny/3, Ny/3))
    end

    lower = [0.0, bounds[1][1], bounds[2][1],  0.0]
    upper = [Inf, bounds[1][2], bounds[2][2], 50]

    # --- Chi² function ---
    function chi2_func(p)
        scale, dx, dy, gamma = p

        model = moffat_2d(
            x_img .- dx .- x_c,
            y_img .- dy .- y_c;
            amplitude = scale,
            alpha = alpha,
            gamma = gamma,
        )

        model = min.(model, sat_level)

        resid = image_clean .- model
        return sum(abs2, resid[combined_mask] ./ uncertainty[combined_mask])
    end

    # --- Initial guess ---
    p0 = [
        initial_scale,
        initial_shift[1],
        initial_shift[2],
        0.65,
    ]

    # --- Optimization ---
    opt = optimize(
        chi2_func,
        lower,
        upper,
        p0,
        Fminbox(LBFGS()),
        Optim.Options(show_trace = false),
        autodiff = :forward
    )

    pbest = Optim.minimizer(opt)
    min_chi2 = Optim.minimum(opt)

    best_scale, best_dx, best_dy, best_gamma = pbest

    # --- Best-fit model ---
    best_model = moffat_2d(
        x_img .- best_dx .- x_c,
        y_img .- best_dy .- y_c;
        amplitude = best_scale,
        alpha = alpha,
        gamma = best_gamma,
    )

    best_model = min.(best_model, sat_level)
    residual = image_clean .- best_model

    return Dict(
        :scale => best_scale,
        :shift => (best_dx, best_dy),
        :alpha => alpha,
        :gamma => best_gamma,
        :chi2 => min_chi2,
        :reduced_chi2 => min_chi2 / dof,
        :model => best_model,
        :residual => residual,
        :success => Optim.converged(opt),
        :n_valid_pixels => n_valid,
        :dof => dof,
    )
end


In [ ]:
function double_moff_model(x, y, params; x_c, y_c, sat_level=62_000)
    scale, dx, dy, alpha, gamma, scale2, alpha2_rel, gamma2_rel = params
    alpha2 = alpha * alpha2_rel
    gamma2 = gamma * gamma2_rel

    xs = x .- dx .- x_c
    ys = y .- dy .- y_c
    model = moffat_2d(xs, ys;
        amplitude = scale,
        alpha = alpha,
        gamma = gamma,
    ) .+ moffat_2d(xs, ys;
        amplitude = scale2,
        alpha = alpha2,
        gamma = gamma2,
    )

    model = min.(model, sat_level)
    return model
end

In [ ]:
function fit_double_moff_psf_chi2(
    image::AbstractMatrix;
    uncertainty = nothing,
    mask = nothing,
    initial_scale::Real = 1e6,
    initial_shift::Tuple{Real,Real} = (0.0, 0.0),
    bounds = nothing,
    sat_level = 62_000,
)

    # --- Handle NaNs ---
    image_clean = copy(image)
    finite_mask = isfinite.(image_clean)

    # --- Combine masks ---
    combined_mask = mask === nothing ? finite_mask : (finite_mask .& .!mask)

    # --- Degrees of freedom ---
    n_valid = count(combined_mask)

    Nx, Ny = size(image_clean)

    # --- Coordinate grids ---
    x = collect(1:Nx)
    y = collect(1:Ny)
    x_img = repeat(x, 1, Ny)
    y_img = repeat(y', Nx, 1)

    x_c = (Nx) / 2
    y_c = (Ny) / 2

    lower = [sat_level, bounds[1][1], bounds[2][1], 0.0, 0.0, 0.0, 0.0, 1.1]
    upper = [Inf, bounds[1][2], bounds[2][2], 10.0, 20.0, Inf, 0.9, 10.0]

    # --- Chi² function ---
    function chi2_func(p)
        model = double_moff_model(x_img, y_img, p; x_c=x_c, y_c=y_c, sat_level=sat_level)
        resid = image_clean .- model
        return sum(abs2, resid[combined_mask] ./ uncertainty[combined_mask])
    end

    # --- Initial guess ---
    p0 = [
        initial_scale,
        initial_shift[1],
        initial_shift[2],
        2.5,
        2.0,
        initial_scale/2,
        0.8, 
        1.5,
    ]

    # --- Optimization ---
    opt = optimize(
        chi2_func,
        lower,
        upper,
        p0,
        Fminbox(LBFGS()),
        Optim.Options(show_trace = false),
        autodiff = :forward
    )

    pbest = Optim.minimizer(opt)
    min_chi2 = Optim.minimum(opt)

    best_scale, best_dx, best_dy, best_alpha, best_gamma, best_scale2, best_rel_alpha, best_rel_gamma = pbest

    best_model = double_moff_model(x_img, y_img, pbest; x_c=x_c, y_c=y_c, sat_level=sat_level)

    best_model = min.(best_model, sat_level)
    residual = image_clean .- best_model

    return Dict(
        :scale => best_scale,
        :scale2 => best_scale2,
        :shift => (best_dx, best_dy),
        :alpha => best_alpha,
        :gamma => best_gamma,
        :alpha2 => best_rel_alpha * best_alpha,
        :gamma2 => best_rel_gamma * best_gamma,
        :chi2 => min_chi2,
        :scale_21 => best_scale2/best_scale,
        :model => best_model,
        :residual => residual,
        :success => Optim.converged(opt),
        :n_valid_pixels => n_valid,
    )
end


In [ ]:
"""
Fit a Moffat PSF to an image cutout by minimizing chi-squared.

Optimized parameters:
(scale, dx, dy, alpha, gamma)
"""
function fit_moff_psf_chi2(
    image::AbstractMatrix;
    uncertainty = nothing,
    mask = nothing,
    initial_scale::Real = 1.0,
    initial_shift::Tuple{Real,Real} = (0.0, 0.0),
    bounds = nothing,
    sat_level = 62_000,
)

    # --- Handle NaNs ---
    image_clean = copy(image)
    finite_mask = isfinite.(image_clean)

    # --- Combine masks ---
    combined_mask = mask === nothing ? finite_mask : (finite_mask .& .!mask)

    # --- Degrees of freedom ---
    n_valid = count(combined_mask)
    n_params = 5
    dof = max(1, n_valid - n_params)

    Ny, Nx = size(image_clean)

    # --- Coordinate grids ---
    x = collect(1:Nx)
    y = collect(1:Ny)
    x_img = repeat(x', Ny, 1)
    y_img = repeat(y, 1, Nx)

    x_c = (Ny) / 2
    y_c = (Nx) / 2

    # --- Bounds ---
    if bounds === nothing
        bounds = ((-Nx/3, Nx/3), (-Ny/3, Ny/3))
    end

    lower = [0.0, bounds[1][1], bounds[2][1], 0.0, 0.0, 0.0, 0.0]
    upper = [Inf, bounds[1][2], bounds[2][2], 100.0, 10.0, 1.0, 100.0]

    # --- Chi² function ---
    function chi2_func(p)
        scale, dx, dy, alpha, gamma, f_gauss, sigma = p

        model = moffat_2d(
            y_img .- dy .- y_c,
            x_img .- dx .- x_c;
            amplitude = scale * (1 - f_gauss),
            alpha = alpha,
            gamma = gamma,
        )
        model += gaussian_2d(
            y_img .- dy .- y_c,
            x_img .- dx .- x_c;
            amplitude = scale * f_gauss,
            sigma = sigma
            )


        model = min.(model, sat_level)

        resid = image_clean .- model
        return sum(abs2, resid[combined_mask] ./ uncertainty[combined_mask])
    end

    # --- Initial guess ---
    p0 = [
        initial_scale,
        initial_shift[1],
        initial_shift[2],
        2.5,
        2.0,
        0.5,
        0.65,
    ]

    # --- Optimization ---
    opt = optimize(
        chi2_func,
        lower,
        upper,
        p0,
        Fminbox(LBFGS()),
        Optim.Options(show_trace = false),
        autodiff = :forward
    )

    pbest = Optim.minimizer(opt)
    min_chi2 = Optim.minimum(opt)

    best_scale, best_dx, best_dy, best_alpha, best_gamma, best_f_gauss, best_sigma = pbest

    # --- Best-fit model ---
    best_model = moffat_2d(
        y_img .- best_dy .- y_c,
        x_img .- best_dx .- x_c;
        amplitude = best_scale,
        alpha = best_alpha,
        gamma = best_gamma,
    )
        
    best_model += gaussian_2d(
            y_img .- best_dy .- y_c,
            x_img .- best_dx .- x_c;
            amplitude = best_scale * best_f_gauss,
            sigma = best_sigma
            )


    best_model = min.(best_model, sat_level)
    residual = image_clean .- best_model

    return Dict(
        :scale => best_scale,
        :shift => (best_dx, best_dy),
        :alpha => best_alpha,
        :gamma => best_gamma,
        :f_gauss => best_f_gauss,
        :sigma => best_sigma,
        :chi2 => min_chi2,
        :model => best_model,
        :residual => residual,
        :success => Optim.converged(opt),
        :n_valid_pixels => n_valid,
    )
end


In [ ]:
function print_fit(fit)
    @printf "fit succeded: %s\n" fit[:success]
    @printf "chi2: %s\n" fit[:chi2]
    @printf "log scale %0.2f\n" log10(fit[:scale])
    # @printf "f_gauss %0.5f\n" fit[:f_gauss]
    @printf "shift %2.1f, %2.1f\n" fit[:shift]...

    for param in [:alpha, :gamma, :ellipticity, :position_angle, :alpha2, :gamma2, :scale2, :scale_21]
        if param in keys(fit)
            @printf "%8s %8.2f\n" param fit[param]
        end
    end

end

In [ ]:
cutout_sizes = [maximum(size(cutout)) for cutout in cutouts]

In [ ]:
bad_idx = eachindex(group_areas)[sortperm(group_areas, rev=true)]
bad_idx = bad_idx[(group_areas[bad_idx] .> 100) .& (cutout_sizes[bad_idx] .> 1)]


In [ ]:
log10(62_000)

In [ ]:
psf = read_image(imgdir * "/psf.fits")


In [ ]:
show_image(psf, log=true)

In [ ]:
zeros

In [ ]:
zeros((3,3))

In [ ]:

colorrange = (-20bkg_rms, 20bkg_rms)
idx_i = 1

idxs = bad_idx[idx_i:idx_i+4]
img_residuals = []
for i in idxs
    cutout = cutouts[i]
    uncert = cutouts_err[i]
    bbox = group_bbox[i]

    bounds = bbox  .+ (-1, 1, -1, 1)
    fit = fit_moff_simple_psf_chi2(cutout, uncertainty=uncert, bounds=(bounds[1:2], bounds[3:4]), alpha=1.1)

    println(bounds, )
    print_fit(fit)
    
    fig = Figure(size=(6*72, 2*72))

    show_image(fig[1,1], cutout, colorrange=colorrange, colormap=:redsblues, )
    nx, ny = size(cutout)
    x1, x2 = nx/2 + bbox[1], nx/2 + bbox[2]
    y1, y2 = ny/2 + bbox[3], ny/2 + bbox[4]
    
    lines!([x1, x2, x2, x1, x1], [y1, y1, y2, y2, y1], color=COLORS[2], )
    scatter!(nx/2 + fit[:shift][1], ny/2 + fit[:shift][2], color=COLORS[2], )


    x0, y0 = nx/2 + fit[:shift][1], ny/2 + fit[:shift][2]
    
    show_image(fig[1,2], fit[:model],  colormap=:redsblues, colorrange=colorrange)
    scatter!(x0, y0, color=COLORS[2])

    show_image(fig[1,3], fit[:residual], colorrange=colorrange, colormap=:redsblues, )

    arc!((x0, y0), 5*fit[:gamma], 0, 2π, color=COLORS[2])

    push!(img_residuals, fit[:residual])

    Colorbar(fig[1,4], colorrange=colorrange ./ bkg_rms, colormap=:redsblues)
    display(fig)
end

In [ ]:

colorrange = (-20bkg_rms, 20bkg_rms)
idx_i = 1

idxs = bad_idx[idx_i:idx_i+10]
img_residuals = []
for i in idxs
    cutout = cutouts[i]
    uncert = cutouts_err[i]
    bbox = group_bbox[i]

    bounds = bbox  .+ (-1, 1, -1, 1)
    fit = fit_moff_psf_chi2(cutout, uncertainty=uncert, bounds=(bounds[1:2], bounds[3:4]))

    println(bounds, )
    print_fit(fit)
    
    fig = Figure(size=(6*72, 2*72))

    show_image(fig[1,1], cutout, colorrange=colorrange, colormap=:redsblues, )
    nx, ny = size(cutout)
    x1, x2 = nx/2 + bbox[1], nx/2 + bbox[2]
    y1, y2 = ny/2 + bbox[3], ny/2 + bbox[4]
    
    lines!([x1, x2, x2, x1, x1], [y1, y1, y2, y2, y1], color=COLORS[2], )
    scatter!(nx/2 + fit[:shift][1], ny/2 + fit[:shift][2], color=COLORS[2], )

    
    show_image(fig[1,2], fit[:model],  colormap=:redsblues, colorrange=colorrange)
    scatter!(nx/2 + fit[:shift][1], ny/2 + fit[:shift][2], color=COLORS[2])

    show_image(fig[1,3], fit[:residual], colorrange=colorrange, colormap=:redsblues, )

    push!(img_residuals, fit[:residual])

    Colorbar(fig[1,4], colorrange=colorrange ./ bkg_rms, colormap=:redsblues)
    display(fig)
end

In [ ]:

colorrange = (-20bkg_rms, 20bkg_rms)
idx_i = 1

idxs = bad_idx[idx_i:idx_i+5]
img_residuals = []
for i in idxs
    cutout = cutouts[i]
    uncert = cutouts_err[i]
    bbox = group_bbox[i]

    bounds = bbox  .+ (-1, 1, -1, 1)
    fit = fit_double_moff_psf_chi2(cutout, uncertainty=uncert, bounds=(bounds[1:2], bounds[3:4]))

    println(bounds, )
    print_fit((fit))
    println(typeof(fit[:model]))
    
    fig = Figure(size=(6*72, 2*72))

    show_image(fig[1,1], cutout, colorrange=colorrange, colormap=:redsblues, )
    nx, ny = size(cutout)
    x1, x2 = nx/2 + bbox[1], nx/2 + bbox[2]
    y1, y2 = ny/2 + bbox[3], ny/2 + bbox[4]
    
    lines!([x1, x2, x2, x1, x1], [y1, y1, y2, y2, y1], color=COLORS[2], )
    scatter!(nx/2 + fit[:shift][1], ny/2 + fit[:shift][2], color=COLORS[2], )

    show_image(fig[1,2], fit[:model],  colormap=:redsblues, colorrange=colorrange)
    scatter!(nx/2 + fit[:shift][1], ny/2 + fit[:shift][2], color=COLORS[2])

    show_image(fig[1,3], fit[:residual], colorrange=colorrange, colormap=:redsblues, )

    push!(img_residuals, fit[:residual])

    Colorbar(fig[1,4], colorrange=colorrange ./ bkg_rms, colormap=:redsblues)
    display(fig)
end

# diffraction spike detection

In [ ]:
for cutout in img_residuals

    fig = Figure()
    
    show_image(fig[1,1], cutout)

    thresh = 3 * bkg_rms

    show_image(fig[1,2], (cutout .> thresh) .| (isnan.(cutout)))
    

    display(fig)
end
    

# Emperical psf model

In [ ]:
using Interpolations

In [ ]:
psf_clean = psf .- median(psf)

In [ ]:
show_image(psf_clean, log=true)

In [ ]:
"""
Fit a PSF image to an image cutout by minimizing chi².

Optimized parameters:
(scale, dx, dy)

dx, dy are shifts in *columns* and *rows*, respectively.
"""
function fit_psf_image_chi2(
    image::AbstractMatrix,
    psf::AbstractMatrix;
    uncertainty = nothing,
    mask = nothing,
    initial_scale::Real = 1e6,
    initial_shift::Tuple{Real,Real} = (0.0, 0.0),
    bounds = nothing,
    sat_level = 62_000,
    method = :NelderMead,
)

    # --- NaN handling ---
    image_clean = copy(image)
    finite_mask = isfinite.(image_clean)

    combined_mask =
        mask === nothing ? finite_mask : (finite_mask .& .!mask)

    n_valid = count(combined_mask)
    n_params = 3
    dof = max(1, n_valid - n_params)

    Nx, Ny = size(image_clean)
    Nx_psf, Ny_psf = size(psf)

    # --- Coordinate centers ---
    cy_img = (Ny ) / 2
    cx_img = (Nx) / 2

    cy_psf = (Ny_psf ) / 2
    cx_psf = (Nx_psf) / 2

    # --- Interpolated PSF ---
    itp = interpolate(psf, BSpline(Cubic(Line(OnGrid()))))
    psf_itp = extrapolate(itp, 0.0)

    # --- Bounds ---
    if bounds === nothing
        bounds = ((-Nx/2, Nx/2), (-Ny/2, Ny/2))
    end

    lower = [sat_level, bounds[1][1], bounds[2][1]]
    upper = [Inf, bounds[1][2], bounds[2][2]]

    # --- χ² function ---
    function chi2_func(p)
        scale, dx, dy = p
        χ2 = 0.0

        @inbounds for col in axes(image_clean, 2), row in axes(image_clean, 1)
            combined_mask[row, col] || continue

            # Map image pixel → PSF coordinates
            psf_row = (row - cx_img) + cx_psf - dx
            psf_col = (col - cy_img) + cy_psf - dy

            model = scale * psf_itp(psf_row, psf_col)
            model = min(model, sat_level)

            δ = (image_clean[row, col] - model) / uncertainty[row, col]
            χ2 += δ * δ
        end

        return χ2
    end

    # --- Initial guess ---
    p0 = [initial_scale, initial_shift[1], initial_shift[2]]

    # --- Optimization ---
    opt = method == :NelderMead ?
        optimize(
            chi2_func,
            lower,
            upper,
            p0,
            Fminbox(NelderMead()),
        ) :
        optimize(
            chi2_func,
            lower,
            upper,
            p0,
            Fminbox(LBFGS()),
            Optim.Options();
            autodiff = :forward,
        )

    pbest = Optim.minimizer(opt)
    min_chi2 = Optim.minimum(opt)

    best_scale, best_dx, best_dy = pbest

    # --- Best-fit model ---
    best_model = zeros(eltype(image_clean), size(image_clean))

    @inbounds for col in axes(best_model, 2), row in axes(best_model, 1)
        psf_row = (row - cy_img) + cx_psf - best_dx
        psf_col = (col - cx_img) + cy_psf - best_dy
        best_model[row, col] =
            min(best_scale * psf_itp(psf_row, psf_col), sat_level)
    end

    residual = image_clean .- best_model

    return Dict(
        :scale => best_scale,
        :shift => (best_dx, best_dy),
        :chi2 => min_chi2,
        :reduced_chi2 => min_chi2 / dof,
        :model => best_model,
        :residual => residual,
        :success => Optim.converged(opt),
        :n_valid_pixels => n_valid,
        :dof => dof,
    )
end


In [ ]:

colorrange = (-20bkg_rms, 20bkg_rms)
idx_i = 1

idxs = bad_idx[idx_i:idx_i+10]
img_residuals = []
for i in idxs
    cutout = cutouts[i]
    uncert = cutouts_err[i]
    bbox = group_bbox[i]

    bounds = bbox  .+ (-1, 1, -1, 1)
    fit = fit_psf_image_chi2(cutout, psf_clean, uncertainty=uncert, bounds=(bounds[1:2], bounds[3:4]))

    println(bounds, )
    # print_fit(fit)
    
    fig = Figure(size=(6*72, 2*72))

    show_image(fig[1,1], cutout, colorrange=colorrange, colormap=:redsblues, )
    nx, ny = size(cutout)
    x1, x2 = nx/2 + bbox[1], nx/2 + bbox[2]
    y1, y2 = ny/2 + bbox[3], ny/2 + bbox[4]
    
    lines!([x1, x2, x2, x1, x1], [y1, y1, y2, y2, y1], color=COLORS[2], )
    scatter!(nx/2 + fit[:shift][1], ny/2 + fit[:shift][2], color=COLORS[2], )

    
    show_image(fig[1,2], fit[:model],  colormap=:redsblues, colorrange=colorrange)
    scatter!(nx/2 + fit[:shift][1], ny/2 + fit[:shift][2], color=COLORS[2])

    show_image(fig[1,3], fit[:residual], colorrange=colorrange, colormap=:redsblues, )

    push!(img_residuals, fit[:residual])

    Colorbar(fig[1,4], colorrange=colorrange ./ bkg_rms, colormap=:redsblues)
    display(fig)
end

# Script version

In [ ]:
using Revise

In [ ]:
include("../analyse_bright_stars.jl")

In [ ]:
img_masked, img_model, img_residual, fits = run_all("../yasone2/img_r_01/", n_max=10)

In [ ]:
clim = (-10bkg_rms, 10bkg_rms)

In [ ]:
show_image(img_masked, colorrange=clim, colormap=:redsblues)

In [ ]:
show_image(img_residual, colorrange=clim, colormap=:redsblues)

In [ ]:
show_image(img_model, colorrange=clim, colormap=:redsblues)

In [ ]:
cens, areas, bboxes =get_group_centroids_areas((img_flags .& 1 .> 0))

In [ ]:
show_image(img_masked, log=true)

In [ ]:
i = 20
cutout = img_masked[get_cutout_bounds(size(img), cens[i], bboxes[i], 20, pad=5)...]

In [ ]:
show_image(cutout)

In [ ]:
fit_moff_simple_psf_chi2(cutout, x_c=size(cutout, 1)/2, y_c=size(cutout, 2)/2, bounds=((-20, 20), (-20, 20)))
    